In [1]:
import random
import os
import shutil
import copy
from skimage import measure
from skimage import morphology as skim
import skimage.io as skio
import warnings
import pandas as pd
import numpy as np
import pathlib
import geopandas as gpd
from rasterio import features, coords
from rasterio.plot import reshape_as_raster
import rasterio
from shapely.geometry import shape
from osgeo import gdal

from cropmask.misc import parse_yaml, make_dirs

from cropmask.preprocess import PreprocessWorkflow
wflow = PreprocessWorkflow("/az-ml-container/configs/preprocess_config_pytest.yaml", 
                                 "/az-ml-container/western_nebraska_landsat_scenes_pytest/LT050320312005011601T1-SC20190418222311/",
                                 "/az-ml-container/external_pytest/nebraska-center-pivots-2005/nbextent-clipped-to-western.geojson")

directory_list = wflow.setup_dirs()

band_list = wflow.yaml_to_band_index()

product_list = wflow.get_product_paths(band_list)

stacked_arr = wflow.load_and_stack_bands(product_list)

wflow.stack_and_save_bands()

In [4]:
shp_frame = gpd.read_file(wflow.source_label_path)

In [5]:
shp_frame = shp_frame.loc[shp_frame.area > wflow.small_area_filter]
shp_frame["geometry"] = shp_frame["geometry"].buffer(wflow.neg_buffer)

In [12]:
is_valid_geom(shp_frame.iloc[638]['geometry'])

TypeError: argument of type 'Polygon' is not iterable

In [13]:
shp_frame.iloc[638]['geometry'].is_empty

True

In [11]:
is_valid_geom(shp_frame.iloc[638]['geometry'].__geo_interface__)

False

In [22]:
shp_frame.geometry.apply(lambda g: is_valid_geom(g.__geo_interface__)).min()

False

In [6]:
# keeps the class of interest if it is there and the polygon of raster extent
meta = wflow.meta.copy()
meta.update({'count':1})
tifname = os.path.splitext(os.path.basename(wflow.source_label_path))[0] + ".tif"
wflow.rasterized_label_path = os.path.join(wflow.NEG_BUFFERED, tifname)
with rasterio.open(wflow.rasterized_label_path, "w+", **wflow.meta) as out:
    out_arr = out.read(1)
    shp_frame = shp_frame.loc[shp_frame.area > wflow.small_area_filter]
    shp_frame["geometry"] = shp_frame["geometry"].buffer(wflow.neg_buffer)
    shp_frame = shp_frame.loc[shp_frame.geometry.is_empty==False]
    # https://gis.stackexchange.com/questions/151339/rasterize-a-shapefile-with-geopandas-or-fiona-python#151861
    shapes = (
        (geom, value)
        for geom, value in zip(shp_frame.geometry, shp_frame.ID)
    )

    burned = features.rasterize(
        shapes=shapes,
        fill=0,
        out_shape=(meta['height'],meta['width']),
        transform=out.transform,
        default_value=1,
    )
    burned[burned < 0] = 0
    burned[burned > 0] = 1
    burned = burned.astype(np.int16, copy=False)
    out.write(burned, 1)

ValueError: Invalid geometry object at index 638

In [7]:
from rasterio.features import is_valid_geom

In [ ]:
shp_frame['geometry'].iloc[3973].is_empty

In [ ]:
    try: 
        img_chip_list = os.listdir(wflow.GRIDDED_IMGS)
        label_chip_list = os.listdir(wflow.GRIDDED_LABELS)
        assert img_chip_list # make sure that this is not empty
        assert label_chip_list
        remove_dirs(directory_list)
    except AssertionError:
        remove_dirs(directory_list)
        print("The product list is empty, check this path: "+ wflow.GRIDDED_IMGS)

In [ ]:
wflow.negative_buffer_and_small_filter(-31, 100)

In [1]:
import random
import os
import shutil
import copy
from skimage import measure
from skimage import morphology as skim
import skimage.io as skio
import warnings
import pandas as pd
import numpy as np
import pathlib
import geopandas as gpd
from rasterio import features, coords
from rasterio.plot import reshape_as_raster
import rasterio
from shapely.geometry import shape
from osgeo import gdal

from cropmask.misc import parse_yaml, make_dirs

from cropmask.preprocess import PreprocessWorkflow
wflow = PreprocessWorkflow("/az-ml-container/configs/preprocess_config_pytest.yaml", 
                                 "/az-ml-container/western_nebraska_landsat_scenes_pytest/LT050320312005011601T1-SC20190418222311/",
                                 "/az-ml-container/external_pytest/nebraska-center-pivots-2005/nbextent-clipped-to-western.geojson")

directory_list = wflow.setup_dirs()

band_list = wflow.yaml_to_band_index()

product_list = wflow.get_product_paths(band_list)

stacked_arr = wflow.load_and_stack_bands(product_list)

wflow.stack_and_save_bands()

wflow.negative_buffer_and_small_filter(-31, 100)

Done applying negbuff of -31 and filtering small labels of area less than 100


True

In [ ]:
def rm_mostly_empty(self, scene_path, label_path):
    """
    Removes a grid that is emptier than the usable data threshold and corrects bad no data value to 0.
    Ignore the User Warning, unsure why it pops up but doesn't seem to impact the array shape. Used because
    very empty grid chips seem to throw off the model by increasing detections at the edges between good data 
    and nodata.
    """

    arr = skio.imread(scene_path)
    arr[arr < 0] = 0
    skio.imsave(scene_path, arr)
    pixel_count = arr.shape[0] * arr.shape[1]
    nodata_pixel_count = (arr == 0).sum()

    if nodata_pixel_count / pixel_count > self.usable_threshold:

        os.remove(scene_path)
        os.remove(label_path)
    print("removed scene and label, {}% bad data".format(self.usable_threshold))

def new_grid_images(self):
    """
    Grids up imagery to a variable size. Filters out imagery with too little usable data.
    appends a random unique id to each tif and label pair, appending string 'label' to the 
    mask.
    """

    ds = gdal.Open(self.stacked_path) # TO DO can get this info earlier and with less file io
    band = ds.GetRasterBand(1)
    xsize = band.XSize
    ysize = band.YSize
    ### TO DO MAKE GRID ID DIRECTORY CREATION HAPPEN IN THE MAKE DIRS FUNCTION, MOVE ALL DIR SETUP TO THAT FUNCTION INSTEAD OF HAVING
    ### IT BE SCATTERED AROUND MULTIPLE FUNCS
    for i in range(0, xsize, self.grid_size):
        for j in range(0, ysize, self.grid_size):
            print(xsize)
            print(ysize)
            print(self.grid_size)
            chip_id = str(i)+'_'+str(j)+'_'+self.scene_id
            self.chip_ids.append(chip_id)
            out_path_img = os.path.join(self.GRIDDED_IMGS, chip_id) + ".tif"
            out_path_label = os.path.join(self.GRIDDED_LABELS, chip_id) + "_label.tif"
            print("image chip file name: {}".format(out_path_img))
            com_string = (
                "gdal_translate -of GTIFF -srcwin "
                + str(i)
                + ", "
                + str(j)
                + ", "
                + str(self.grid_size)
                + ", "
                + str(self.grid_size)
                + " "
                + str(self.stacked_path)
                + " "
                + str(out_path_img)
            )
            os.system(com_string)
            com_string = (
                "gdal_translate -of GTIFF -srcwin "
                + str(i)
                + ", "
                + str(j)
                + ", "
                + str(self.grid_size)
                + ", "
                + str(self.grid_size)
                + " "
                + str(self.rasterized_label_path)
                + " "
                + str(out_path_label)
            )
            os.system(com_string)
            self.rm_mostly_empty(out_path_img, out_path_label)
    return True # for testing to confirm it worked

In [ ]:

def compute(unsigned char[:, :, :] input):
    """reverses bands inefficiently

    Given input and output uint8 arrays, fakes an CPU-intensive
    computation.
    """
    cdef int I, J, K
    cdef int i, j, k, l
    cdef double val
    I = input.shape[0]
    J = input.shape[1]
    K = input.shape[2]
    output = np.empty((I, J, K), dtype='uint8')
    cdef unsigned char[:, :, :] output_view = output
    with nogil:
        for i in range(I):
            for j in range(J):
                for k in range(K):
                    val = <double>input[i, j, k]
                    for l in range(2000):
                        val += 1.0
                    val -= 2000.0
                    output_view[~i, j, k] = <unsigned char>val
    return output

In [ ]:
import concurrent.futures

import rasterio
from rasterio._example import compute


def main(infile, outfile, num_workers=4):
    """Process infile block-by-block and write to a new file

    The output is the same as the input, but with band order
    reversed.
    """

    with rasterio.Env():

        with rasterio.open(infile) as src:

            # Create a destination dataset based on source params. The
            # destination will be tiled, and we'll process the tiles
            # concurrently.
            profile = src.profile
            profile.update(blockxsize=128, blockysize=128, tiled=True)

            with rasterio.open(outfile, "w", **profile) as dst:

                # Materialize a list of destination block windows
                # that we will use in several statements below.
                windows = [window for ij, window in dst.block_windows()]

                # This generator comprehension gives us raster data
                # arrays for each window. Later we will zip a mapping
                # of it with the windows list to get (window, result)
                # pairs.
                data_gen = (src.read(window=window) for window in windows)

                with concurrent.futures.ThreadPoolExecutor(
                    max_workers=num_workers
                ) as executor:

                    # We map the compute() function over the raster
                    # data generator, zip the resulting iterator with
                    # the windows list, and as pairs come back we
                    # write data to the destination dataset.
                    for window, result in zip(
                        windows, executor.map(compute, data_gen)
                    ):
                        dst.write(result, window=window)